Per consentire al motore di ricerca di lavorare su informazioni strutturate, ogni giocatore viene trasformato in un documento testuale che descrive le sue caratteristiche principali. Questa rappresentazione include ruolo, percentuali di tiro, efficienza ai liberi, capacità di creazione offensiva, rimbalzi, difesa, metriche avanzate e ulteriori annotazioni qualitative. Un documento può dunque assumere una forma come: “Guardia, 40% da tre, 88% ai liberi, 6 rimbalzi, buon difensore, ottima efficienza offensiva”. L’intero insieme dei giocatori costruisce il corpus testuale che sarà in seguito indicizzato.

In [ ]:
import os
import pandas as pd
import numpy as np

# ============================
# 1. Config & caricamento dati
# ============================

# Path al dataset pulito della PARTE 1
# Cambia il path se diverso nel tuo progetto
DATA_PATH = "../data/drafted_cleaned.csv"

df = pd.read_csv(DATA_PATH)

print("Shape dataset:", df.shape)
print(df.columns)


# ===================================
# 2. Funzioni helper per descrizioni
# ===================================

def safe_get(row, col, default=None):
    """Restituisce row[col] se esiste e non è NaN, altrimenti default."""
    if col in row and not pd.isna(row[col]):
        return row[col]
    return default

def clean_stat(value):
    """
    Converte -1 e -100 (valori sentinella del dataset) in None.
    Lascia invariati 0, 0.0 o altri numeri validi.
    """
    if value is None or pd.isna(value):
        return None
    try:
        v = float(value)
    except:
        return None

    # Valori sentinella del tuo dataset
    if v in (-1, -100):
        return None

    return v


def normalize_percentage(v):
    """
    Normalizza percentuali NBA in modo sicuro.
    Gestisce:
    - -100, -1, 0 e 1 come missing quando non veri percentage
    - valori decimali (0.35 → 35)
    - valori interi (35 → 35)
    - valori erronei (350 → 35)
    """
    try:
        v = float(v)
    except:
        return None

    # ---- valori placeholder da ignorare ----
    if v in (-100, -1, 1):
        return None

    # ---- valore già decimale, es: 0.35 ----
    if 0 < v < 1:
        return v * 100

    # ---- valore già percentuale "normale" ----
    if 1 < v <= 100:
        return v

    # ---- errori come 350 → 35 ----
    if 100 < v < 1000:
        return v / 10

    # ---- casi estremi (es 2300 → 23) ----
    if v >= 1000:
        return v / 100

    return None

for col in ["3P%", "FT%"]:
    if col in df.columns:
        df[col] = df[col].apply(normalize_percentage)

df.to_csv("../data/drafted_cleaned.csv", index=False)
print("Salvato drafted_cleaned.csv normalizzato.")




def format_percent(value, decimals=1):
    v = normalize_percentage(value)
    if v is None:
        return None
    return f"{v:.{decimals}f}%"


def bucketize_3p(pct):
    v = normalize_percentage(pct)
    if v is None:
        return None

    if v >= 40:
        return "ottimo tiratore da 3 punti"
    elif v >= 35:
        return "buon tiratore da 3 punti"
    elif v >= 30:
        return "tiratore da 3 punti discreto"
    else:
        return "tiratore da 3 punti poco affidabile"

def bucketize_ft(pct):
    v = normalize_percentage(pct)
    if v is None:
        return None

    if v >= 85:
        return "eccellente tiratore di tiri liberi"
    elif v >= 75:
        return "buon tiratore di tiri liberi"
    return "tiratore di liberi migliorabile"


def bucketize_reb(reb):
    """Aggettivo per rimbalzi totali."""
    if reb is None:
        return None
    try:
        v = float(reb)
    except (TypeError, ValueError):
        return None
    if v >= 10:
        return "forte rimbalzista"
    elif v >= 7:
        return "buon rimbalzista"
    elif v >= 4:
        return "rimbalzista discreto"
    else:
        return "rimbalzista non di impatto"

def bucketize_ast(ast):
    """Aggettivo per assist."""
    if ast is None:
        return None
    try:
        v = float(ast)
    except (TypeError, ValueError):
        return None
    if v >= 9:
        return "ottimo playmaker e passatore"
    elif v >= 6:
        return "buon passatore"
    else:
        return "non particolarmente orientato agli assist"

def bucketize_scorer(pts):
    """Aggettivo per capacità realizzativa (punti a partita)."""
    if pts is None:
        return None
    try:
        v = float(pts)
    except (TypeError, ValueError):
        return None
    if v >= 25:
        return "realizzatore di altissimo livello"
    elif v >= 18:
        return "buon realizzatore"
    elif v >= 12:
        return "realizzatore discreto"
    else:
        return "giocatore non focalizzato sulla realizzazione"


# =======================================
# 3. Funzione principale: riga -> testo
# =======================================

def build_player_text(row):
    pieces = []

    # ============================
    #  Nome e provenienza
    # ============================
    name = safe_get(row, "Player")
    if name:
        pieces.append(f"Giocatore: {name}.")

    college = safe_get(row, "College")
    if college:
        pieces.append(f"Proveniente da {college}.")

    # ============================
    #  Draft info
    # ============================
    draft_year = safe_get(row, "DraftYear")
    pick = safe_get(row, "Pick")
    pickband = safe_get(row, "PickBand")
    if draft_year and pick:
        pieces.append(
            f"Selezionato al Draft NBA {int(draft_year)} con la pick {int(pick)} ({pickband})."
        )

    # ============================
    #  Status
    # ============================
    status = safe_get(row, "Status")
    if status == "Active":
        pieces.append("Giocatore attualmente in attività NBA.")
    elif status == "Retired":
        pieces.append("Giocatore ritirato dalla NBA.")

    # ============================
    #  Tiro da 3 con soglie aggiornate
    # ============================
    raw_p3 = safe_get(row, "3P%")
    p3 = normalize_percentage(raw_p3)
    if p3 is not None:
        bucket_3 = bucketize_3p(p3)
        pieces.append(f"Tiro da 3: {p3:.1f}%.")
        if bucket_3:
            pieces.append(f"{bucket_3}.")

    # ============================
    #  Tiri liberi con soglie aggiornate
    # ============================
    raw_ft = safe_get(row, "FT%")
    ft = normalize_percentage(raw_ft)
    if ft is not None:
        bucket_ft_s = bucketize_ft(ft)
        pieces.append(f"Tiri liberi: {ft:.1f}%.")
        if bucket_ft_s:
            pieces.append(f"{bucket_ft_s}.")

    # ============================
    #  Statistiche per partita
    # ============================
    # Punti
    pts_pg = clean_stat(safe_get(row, "PTS.1"))
    if pts_pg is not None:
        bucket_pts_s = bucketize_scorer(pts_pg)
        pieces.append(f"Segna {pts_pg} punti a partita.")
        if bucket_pts_s:
            pieces.append(f"{bucket_pts_s}.")

    # Rimbalzi
    reb_pg = clean_stat(safe_get(row, "TRB.1"))
    if reb_pg is not None:
        bucket_reb_s = bucketize_reb(reb_pg)
        pieces.append(f"Cattura {reb_pg} rimbalzi a partita.")
        if bucket_reb_s:
            pieces.append(f"{bucket_reb_s}.")

    # Assist
    ast_pg = clean_stat(safe_get(row, "AST.1"))
    if ast_pg is not None:
        bucket_ast_s = bucketize_ast(ast_pg)
        pieces.append(f"Distribuisce {ast_pg} assist a partita.")
        if bucket_ast_s:
            pieces.append(f"{bucket_ast_s}.")

    # ============================
    #  Statistiche avanzate
    # ============================
    ws = clean_stat(safe_get(row, "WS"))
    if ws is not None:
        pieces.append(f"Win Shares totali: {ws}.")

    bpm = clean_stat(safe_get(row, "BPM"))
    if bpm is not None:
        pieces.append(f"Box Plus/Minus: {bpm}.")

    vorp = clean_stat(safe_get(row, "VORP"))
    if vorp is not None:
        pieces.append(f"VORP: {vorp}.")

    # Expected WS e Delta
    expected_ws = safe_get(row, "Expected_WS")
    delta = safe_get(row, "Delta")
    if expected_ws is not None:
        pieces.append(f"Valore atteso: {expected_ws:.1f} WS.")
    if delta is not None:
        pieces.append(f"Scostamento rispetto alle aspettative: {delta:.1f} WS.")

    # Chiusura
    pieces.append("Profilo generato automaticamente dal sistema di analisi del Draft NBA.")

    return " ".join(pieces)



# ======================================
# 4. Applicazione al DataFrame & salvat.
# ======================================

# 1. Generazione del profilo testuale
df["text_profile"] = df.apply(build_player_text, axis=1)

# 2. Preview
for i in range(5):
    print("==== Giocatore", i, "====")
    print(df.iloc[i]["text_profile"])
    print()

# 3. Costruzione corpus
corpus_cols = ["Player", "DraftYear", "Pick", "PickBand", "text_profile", "Status"]
corpus_cols = [col for col in corpus_cols if col in df.columns]

corpus_df = df[corpus_cols].copy()

# ID univoco
corpus_df.insert(0, "player_id", corpus_df.index.astype(int))

# 4. Salvataggio
os.makedirs("../data/processed", exist_ok=True)

corpus_df.to_json("../data/processed/nba_players_corpus.json",
                  orient="records", force_ascii=False, indent=2)

corpus_df.to_csv("../data/processed/nba_players_corpus.csv", index=False)

print("Corpus salvato in:")
print(" - ../data/processed/nba_players_corpus.json")
print(" - ../data/processed/nba_players_corpus.csv")


Shape dataset: (8323, 36)
Index(['DraftYear', 'Pick', 'Tm', 'Player', 'College', 'Seasons', 'Games',
       'MP', 'PTS', 'TRB', 'AST', 'FG%', '3P%', 'FT%', 'MP.1', 'PTS.1',
       'TRB.1', 'AST.1', 'WS', 'WS/48', 'BPM', 'VORP', 'Forfeited', 'PickBand',
       'Debut', 'WS_available', 'WS/48_available', 'BPM_available',
       'VORP_available', 'MP_available', 'TRB_available', 'FG%_available',
       'FT%_available', 'MP.1_available', 'TRB.1_available', 'Status'],
      dtype='object')
Salvato drafted_cleaned.csv normalizzato.
==== Giocatore 0 ====
Giocatore: Clifton McNeely. Proveniente da Texas Wesleyan University. Selezionato al Draft NBA 1947 con la pick 1 (Top10). Giocatore ritirato dalla NBA. Segna 0.0 punti a partita. giocatore non focalizzato sulla realizzazione. Cattura 0.0 rimbalzi a partita. rimbalzista non di impatto. Distribuisce 0.0 assist a partita. non particolarmente orientato agli assist. Win Shares totali: 0.0. Box Plus/Minus: 0.0. VORP: 0.0. Profilo generato automati